In [33]:
import gensim
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import Doc2Vec
from gensim.models.doc2vec import LabeledSentence
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import re

In [52]:
from sklearn import cross_validation
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier as RFC
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import pickle
from sklearn.model_selection import train_test_split

In [2]:
with open('us_trial.text', encoding='utf-8') as textfile:
    sentences = textfile.readlines()

with open('us_trial.labels', encoding='utf-8') as textfile:
    emoji = textfile.readlines()

In [25]:
tweets = []
s = []
t = []
for sent, emj in zip(sentences,emoji):
    s.append(sent)
    t.append(int(emj))
tweets.append(('sentences', s))
tweets.append(('emoji', t))

def make_df(tweets):
    df_c = pd.DataFrame.from_items(tweets)
    df = df_c.loc[:,["sentences","emoji"]]
    df.dropna(how="any", inplace=True)   
    #print(df.head)
    return df

In [26]:
df = make_df(tweets)

In [31]:
def sampling_dataset(df):
    count = 5000
    class_df_sampled = pd.DataFrame(columns = ["sentences","emoji"])
    temp = []
    for c in df:
        class_indexes = df[c].index
        random_indexes = np.random.choice(class_indexes, count, replace=False)
        temp.append(df.loc[random_indexes])
        
    for each_df in temp:
        class_df_sampled = pd.concat([class_df_sampled,each_df],axis=0)
    
    return class_df_sampled

df = sampling_dataset(df)
df.reset_index(drop=True,inplace=True)
print (df.head())
print (df.shape)

                                           sentences emoji
0  Elise has really changed in 3 years!! @ Kill D...    16
1  Sunday Sushi @user 4:30pm - 8:30pm come see me...    16
2  AMAZING FATHERS DAY! Got to spend it with my l...     0
3  Daydreamin' #SecretStairs #MichelTorena #Silve...    13
4  This guy . Hanging out in castles 'n shit #sou...     1
(10000, 2)


In [40]:
lmtzr = WordNetLemmatizer()
w = re.compile("\w+",re.I)

def label_sentences(df):
    labeled_sentences = []
    for index, datapoint in df.iterrows():
        tokenized_words = re.findall(w,datapoint["sentences"].lower())
        labeled_sentences.append(LabeledSentence(words=tokenized_words, tags=['SENT_%s' %index]))
    return labeled_sentences

def train_doc2vec_model(labeled_sentences):
    model = Doc2Vec(alpha=0.025, min_alpha=0.025)
    model.build_vocab(labeled_sentences)
    for epoch in range(10):
        model.train(labeled_sentences, total_examples=model.corpus_count, epochs=model.iter)
        model.alpha -= 0.002 
        model.min_alpha = model.alpha
    
    return model

sen = label_sentences(df)
model = train_doc2vec_model(sen)

In [41]:
def vectorize_comments(df,d2v_model):
    y = []
    comments = []
    for i in range(0,df.shape[0]):
        label = 'SENT_%s' %i
        comments.append(d2v_model.docvecs[label])
    df['vectorized_comments'] = comments
    
    return df

df = vectorize_comments(df,model)
print (df.head(2))

                                           sentences emoji  \
0  Elise has really changed in 3 years!! @ Kill D...    16   
1  Sunday Sushi @user 4:30pm - 8:30pm come see me...    16   

                                 vectorized_comments  
0  [0.094514, 0.367859, -0.140067, -0.255143, -0....  
1  [-0.016482, 0.173833, -0.0960274, -0.109512, 0...  


In [67]:
def train_classifier(X,y):
    n_estimators = [200,400]
    min_samples_split = [2]
    min_samples_leaf = [1]
    bootstrap = [True]

    parameters = {'n_estimators': n_estimators, 'min_samples_leaf': min_samples_leaf,
                  'min_samples_split': min_samples_split}

    clf = GridSearchCV(RFC(verbose=1,n_jobs=4), cv=4, param_grid=parameters)
    clf.fit(X, y)
    return clf

#print(len(df['emoji']))


X_train, X_test, y_train, y_test = train_test_split(df["vectorized_comments"].tolist(), df['emoji'], test_size=0.3, random_state=33)

#print(df["vectorized_comments"].tolist())
#classifier = train_classifier(X_train,y_train)
#print (classifier.best_score_)
#print (classifier.score(X_test,y_test))

In [69]:
clf_logreg = linear_model.LogisticRegression(random_state = 42)
clf_logreg.fit(X_train, y_train)


ValueError: Unknown label type: 'unknown'